In [14]:
%load_ext dotenv
%dotenv ../../src/.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [15]:
import sys
sys.path.append("../../src")

In [16]:
from utils.logger import get_logger
_logs = get_logger(__name__)
_logs.info("Notebook started successfully")

2025-11-12 00:39:32,852, 2927690305.py, 3, INFO, Notebook started successfully
